## Dataset characteristics:


##### Import data for visualisations

In [1]:
import profiles

Models: 
XGBoost train score: 0.9959
XGBoost valid score: 0.8396


Logistic regression train score: 0.8122
Logistic regression valid score: 0.8216


Dataset bias: 19.96% of positive answers 



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 69 columns):
AGE                                10000 non-null float64
AGRGT_BAL_ALL_XCLD_MRTG            10000 non-null float64
AUTO_2_OPEN_DATE_YRS               10000 non-null float64
AUTO_HI_CRDT_2_ACTUAL              10000 non-null float64
AVG_BAL_ALL_FNC_REV_ACTS           10000 non-null float64
AVG_BAL_ALL_PRM_BC_ACTS            10000 non-null float64
D_DEPTCARD                         10000 non-null float64
D_NA_AVG_BAL_ALL_FNC_REV_ACTS      10000 non-null float64
D_NA_M_SNCOLDST_BNKINSTL_ACTOPN    10000 non-null float64
D_NA_M_SNC_MST_RCNT_ACT_OPN        10000 non-null float64
D_NA_M_SNC_MST_RCNT_MRTG_DEAL      10000 non-null float64
D_NA_M_SNC_OLDST_MRTG_ACT_OPN      10000 non-null float64
D_

This is an uplift modelling dataset (package __Information__ from R), with indicatior variable "TREATMENT" as a column (prepared for single-model approach)
Data is highly biased, so above scores dont seem to be good (logistic regression have score close to answearing always __no__). Other thing is, column names lack explanation itself, so i will try to focus on columns which names are clear to understand.

Basing on previous explanations using lime, from 69 parameters I've chosen to focus on following 11 explainatory variables, which occured as most important.
1. "TREATMENT" - control grop indicator, 1 if client was engaged in marketing campaign 
1. "AGE" - age
1. "N_OPEN_REV_ACTS" - number of open revolving accounts
1. "D_REGION_A" - indicator variable, if clinet is from region A
1. "MRTG_1_MONTHLY_PAYMENT" - mortage monthy payment
1. "N_DISPUTED_ACTS" - number of disputed accounts
1. "TOT_HI_CRDT_CRDT_LMT" - sum of credit limits
1. "HI_RETAIL_CRDT_LMT" - retail credit limit
1. "PREM_BANKCARD_CRED_LMT", - premium credit card limit
1. "AVG_BAL_ALL_FNC_REV_ACTS" - average balance of all financial revolving accounts
1. "M_SNCOLDST_OIL_NTN_TRD_OPN" - unfortunately, theres no explanation for this parameter

## Observation 11 explanation

![1](plot11_1.png)
![2](plot11_2.png)
![3](plot11_3.png)

### XGBoost model

This is the rare case of model predicting right answear of PURCHASE, taking into consideration structure of the dataset. Previous analysis concluted, that this client was correctly predicted to purchase, because of his wealth, and despite his mortage payment, beeing its second most important variable. However, most important variable N_OPEN_REV_ACTS=27 as we see on the plot, has completly flat neighborhood, so its only important that its bigger then 10. Also, slight non-monotonicity in its plot before 10 seems to be an effect of an overfit. Other plots do not suggest great influence on the model, considering TREATMENT as well. Summarising, model suggests, that this client is wealthy and will buy anyway, even decreasing its probability after TREATMENT. 

### Linear model

As we see, in most cases linear model is approximating XGBoost behaviour in a linear manner (N_OPEN_REV_ACTS), however PREM_BANKCARD_CRED_LMT and MRTG_1_MONTHLY_PAYMENT seem to be a great trouble, suggesting that most train obsarvations were centered around small values with decreasing trend, and model extrapolated it onto bigger values, what led to wrong answears.

## Observation 2 explanation

![1](plot2_1.png)
![2](plot2_2.png)
![3](plot2_3.png)

### XGBoost model

In this case, gradient boosting model is sure that this is a clear "not buying" case. All plots look completly flat, and nothing can change this answear.

### Linear model

Due to linear model only parameters that could change this situation are higher N_OPEN_REV_ACTS and TOT_HI_CRDT_CRDT_LMT, but considering how boosting model works, it is probably an overfit to the most important parameters. 

## False negative example explanation

![1](plotfn_1.png)
![2](plotfn_2.png)
![3](plotfn_3.png)

### XGBoost model

This is a very rare case of false-negative prediction by gradient boosting model (considering that positive answears make up 80% of the dataset). All explanations seem to flat around the predictions (omitting PREM_BANKCARD_CRED_LMT=0), suggesting that model prediction is stable. Here hovewer, TREATMENT can positively influence the prediction, increasing it by about 10 percentage points, condratictory to the first case of "sleeping dog" observation 11.

### Linear model

Linear model of course do not have this false negative, because of answearing mostly always no. Explanations do not  seem to be much different.